In [1]:
from pyiron_base import Project, job
from pyiron_base.project.delayed import DelayedObject

In [2]:
from inspect import isfunction

In [3]:
def get_kwargs(lst):
    return {t['targetHandle']: {'source': t['source'], 'sourceHandle': t['sourceHandle']} for t in lst}

In [4]:
def group_edges(edges_lst):
    edges_sorted_lst = sorted(edges_lst, key=lambda x: x['target'], reverse=True)     
    total_lst, tmp_lst = [], []
    target_id = edges_sorted_lst[0]['target'] 
    for ed in edges_sorted_lst:
        if target_id == ed["target"]:
            tmp_lst.append(ed)
        else:
            total_lst.append((target_id, get_kwargs(lst=tmp_lst)))
            target_id = ed["target"]
            tmp_lst = [ed]
    total_lst.append((target_id, get_kwargs(lst=tmp_lst)))
    return total_lst

In [5]:
def get_source_handles(edges_lst):
    source_handle_dict = {}
    for ed in edges_lst: 
        if ed['source'] not in source_handle_dict.keys():
            source_handle_dict[ed['source']] = [ed['sourceHandle']]
        else:
            source_handle_dict[ed['source']].append(ed['sourceHandle'])
    return source_handle_dict

In [6]:
def get_source(nodes_dict, delayed_object_dict, source, sourceHandle):
    if source in delayed_object_dict.keys():
        return delayed_object_dict[source].__getattr__("output").__getattr__(sourceHandle)
    else:
        return nodes_dict[source]

In [7]:
def get_delayed_object_dict(total_lst, nodes_dict, source_handle_dict, pyiron_project):
    delayed_object_dict = {}
    for item in total_lst: 
        key, input_dict = item
        kwargs = {
            k: get_source(
                nodes_dict=nodes_dict, 
                delayed_object_dict=delayed_object_dict, 
                source=v["source"], 
                sourceHandle=v["sourceHandle"],
            ) 
            for k, v in input_dict.items()
        }
        delayed_object_dict[key] = job(
            funct=nodes_dict[key],
            output_key_lst=source_handle_dict.get(key, []),
        )(**kwargs, pyiron_project=pyiron_project)
    return delayed_object_dict

In [8]:
def add_x_and_y(x, y):
    z = x + y
    return {"x": x, "y": y, "z": z}

In [9]:
def add_x_and_y_and_z(x, y, z):
    w = x + y + z
    return w

In [10]:
nodes_dict = {
    0: add_x_and_y_and_z,
    1: add_x_and_y,
    2: 1,
    3: 2,
}

In [11]:
edges_lst = [
    {'target': 0, 'targetHandle': 'x', 'source': 1, 'sourceHandle': 'x'},
    {'target': 1, 'targetHandle': 'x', 'source': 2, 'sourceHandle': None},
    {'target': 1, 'targetHandle': 'y', 'source': 3, 'sourceHandle': None},
    {'target': 0, 'targetHandle': 'y', 'source': 1, 'sourceHandle': 'y'},
    {'target': 0, 'targetHandle': 'z', 'source': 1, 'sourceHandle': 'z'},
]

In [12]:
nodes_dict, edges_lst

({0: <function __main__.add_x_and_y_and_z(x, y, z)>,
  1: <function __main__.add_x_and_y(x, y)>,
  2: 1,
  3: 2},
 [{'target': 0, 'targetHandle': 'x', 'source': 1, 'sourceHandle': 'x'},
  {'target': 1, 'targetHandle': 'x', 'source': 2, 'sourceHandle': None},
  {'target': 1, 'targetHandle': 'y', 'source': 3, 'sourceHandle': None},
  {'target': 0, 'targetHandle': 'y', 'source': 1, 'sourceHandle': 'y'},
  {'target': 0, 'targetHandle': 'z', 'source': 1, 'sourceHandle': 'z'}])

In [13]:
pr = Project("test")

In [14]:
pr.remove_jobs(recursive=True, silently=True)

0it [00:00, ?it/s]

In [15]:
delayed_object_dict = get_delayed_object_dict(
    total_lst=group_edges(edges_lst), 
    nodes_dict=nodes_dict, 
    source_handle_dict=get_source_handles(edges_lst), 
    pyiron_project=pr
)
delayed_object_dict

{1: <pyiron_base.project.delayed.DelayedObject at 0x13f284c20>,
 0: <pyiron_base.project.delayed.DelayedObject at 0x13f284ef0>}

In [16]:
delayed_object_dict[list(delayed_object_dict.keys())[-1]].pull()

The job add_x_and_y_1a46b923e8b2e71cc1b60655a4852ea7 was saved and received the ID: 14
The job add_x_and_y_and_z_b303b96668df411d6b51eb1d5f1eeb18 was saved and received the ID: 15


6

In [17]:
pr.job_table()

,id,status,chemicalformula,job,subjob,projectpath,project,timestart,timestop,totalcputime,computer,hamilton,hamversion,parentid,masterid
0,14,finished,None,add_x_and_y_1a46b923e8b2e71cc1b60655a4852ea7,/add_x_and_y_1a46b923e8b2e71cc1b60655a4852ea7,None,/Users/janssen/notebooks/2024/2024-11-21-python-workflow-definition/test/,2024-11-21 17:02:42.449017,None,None,pyiron@mac.mpie.de#1,PythonFunctionContainerJob,0.4,None,None
1,15,finished,None,add_x_and_y_and_z_b303b96668df411d6b51eb1d5f1eeb18,/add_x_and_y_and_z_b303b96668df411d6b51eb1d5f1eeb18,None,/Users/janssen/notebooks/2024/2024-11-21-python-workflow-definition/test/,2024-11-21 17:02:42.570598,None,None,pyiron@mac.mpie.de#1,PythonFunctionContainerJob,0.4,None,None
